In [ ]:
%pip install pymysql
%pip install sshtunnel
%pip install pandas

In [2]:
import pymysql
from sshtunnel import SSHTunnelForwarder
import paramiko
from os.path import expanduser
import pandas as pd

In [3]:
import numpy as np

In [4]:
import time
import matplotlib.pyplot as plt
import math

In [5]:
home = expanduser('~')
mypkey = paramiko.RSAKey.from_private_key_file( home + "/.ssh/id_rsa")

sql_hostname = '127.0.0.1'
sql_username = 'readonly'
sql_password = None
sql_main_database = 'ML3_mirror'
sql_port = 3306
ssh_host = 'flagon.cs.umn.edu'
ssh_user = 'kanna128'
ssh_port = 22
sql_ip = '1.1.1.1.1'

In [6]:
def draw_graph(values, labels, max_vals, scale):
  bars = plt.bar(range(len(values[1])), values[1], color=(0.2, 0.4, 0.6, 0.6))

  if((isinstance(scale[1], int))):
    for bar in bars:
      yval = bar.get_height()
      plt.text(bar.get_x() + 0.5*bar.get_width(), yval + values[2], yval)

  xticks = []
  yticks = []

  if(scale[0] != 0):
    xmax = max_vals[0]
    xscale = scale[0]
    for k in np.arange(0, xmax, xscale):
      xticks.append(k)
    plt.xticks(xticks)
  else:
    plt.xticks(range(len(values[0])), values[0])
  
  ymax = max_vals[1]
  yscale = scale[1]

  for k in np.arange(0, ymax, yscale):
    yticks.append(k)
  plt.yticks(yticks)

  plt.xlabel(labels[0])
  plt.ylabel(labels[1])
  plt.ylim(0,max_vals[1])

  if(max_vals[0] != 0):
    plt.xlim(0, max_vals[0])
  
  plt.show()

In [6]:
BASELINE = "{\"engineId\":\"baseline\"}"
ITEM_ITEM = "{\"engineId\":\"item-item\"}"
SVD = "{\"engineId\":\"svd\"}"
PICK_GROUPS = "{\"engineId\":\"pick-groups\"}"

LONG_TO_SHORT = {BASELINE: "baseline", ITEM_ITEM: "item-item", SVD:"svd", PICK_GROUPS: "pick-groups"}
SHORT_TO_LONG = {"baseline": BASELINE, "item-item":ITEM_ITEM, "svd":SVD, "pick-groups":PICK_GROUPS}

In [42]:
########################################### PRIMARY_DICTS #######################################################

def primary_dicts(users, change_events, fields, tstamps_dict):
    
    users_count = users.shape[0]
    uid = fields[0]
    begin = fields[1]
    
    pattern_list_dict = {} # dict to store key=userId, val=curralg in the iteration process
    # tstamp_list_dict = {}

    # tstamp_list_dict = users.set_index('userId').to_dict()['group']

    # user_id_keys = users[uid].to_list()
    # tstamp_list_dict = dict.fromkeys(user_id_keys, 0)
    tstamps_list_dict = {}
    # pattern_list_dict = users.set_index('userId').to_dict()['group']

    for k in range(users_count):
        userId = users.at[k, uid]
        first_item = [ SHORT_TO_LONG[users.at[k, begin]] ] # following the format of log_action.log_Json for future comparisons
        pattern_list_dict[userId] = first_item
        tstamps_list_dict[userId] = 0

    changes_dict = {} # needed to later calculate median changes per user
    changes_count = change_events.shape[0]
    cmp_field = fields[2]
    tstamp_field = fields[3]

    other_users = set()
    # loop to count number of changes correctly; comparing next with prev algorithm
    for index, row in change_events.iterrows():
      userId = row[uid]
      item_to_cmp = row[cmp_field]

      if(pattern_list_dict[userId][-1] != item_to_cmp):
        changes_dict[userId] = 1 + changes_dict.get(userId, 0)
        
        diff = row[tstamp_field] - tstamps_dict[userId]
        tstamps_list_dict[userId] = diff.total_seconds()
        # if diff.total_seconds()<3600:
        #   tstamp_list_dict[userId] = diff.total_seconds()
        # if diff.total_seconds()>3600:
        #   other_users.add(userId)

        # tstamps_sum_dict[userId] += diff.total_seconds()
        # tstamps_dict[userId] = row[tstamp_field]
      
        pattern_list_dict[userId].append(item_to_cmp)
    
    # for i in range(changes_count):
    #   userId = change_events.at[i, uid]
    #   item_to_cmp = change_events.at[i, cmp_field]
    #   if(pattern_list_dict[userId][-1] != item_to_cmp):
    #     changes_dict[userId] = 1 + changes_dict.get(userId, 0)
    #     # if userId in user_id_keys:
    #     # tstamp_list_dict[userId] = [ change_events.at[i, tstamp_field] ]
        
    #     diff = change_events.at[i, tstamp_field] - tstamps_dict[userId]
    #     # tstamps_sum_dict[userId] += diff.total_seconds()
    #     # tstamps_dict[userId] = change_events.at[i, tstamp_field]

    #     # # tstamp_list_dict[userId].append(diff.total_seconds())
    #     if diff.total_seconds()<3600 :
    #       tstamps_list_dict[change_events.at[i, uid]] = diff.total_seconds()
    #     if diff.total_seconds()>3600:
    #       # tstamp_list_dict[userId].append(diff.total_seconds())
    #       other_users.add(userId)
    #     pattern_list_dict[userId].append(item_to_cmp)
    
    # tstamp_list_dict = {key:val for key,val in tstamp_list_dict.items() if val!=0}
    # tstamps_list_dict = {key:val for key,val in tstamps_list_dict.items() if key not in other_users}

    tstamps_list_dict = {key:val for key,val in tstamps_list_dict.items() if val>0 and val<3600}
    tstamps_list_dict = {key:val for key,val in tstamps_list_dict.items() if changes_dict[key] < 3}
    
    return (changes_dict, tstamps_list_dict, pattern_list_dict)

################################################## FIG_2 #######################################################

def fig_2(recchange_dict, changes_dict):
    # count_dict: key=algorithm, val=(no. of users initially assigned, no. of users initially assigned and switched at least once)
    count_dict = {"baseline":[0,0], "item-item":[0,0], "svd":[0,0], "pick-groups":[0,0]}
    for userId,recalg_list in recchange_dict.items():
        recalg = LONG_TO_SHORT[recalg_list[0]]
        count_dict[recalg][0] +=1
        if userId in changes_dict.keys():
            count_dict[recalg][1] += 1
    
    baseline = count_dict["baseline"][1]/count_dict["baseline"][0]
    item_item = count_dict["item-item"][1]/count_dict["item-item"][0]
    svd = count_dict["svd"][1]/count_dict["svd"][0]

    values = [["baseline", "item-item", "svd"], [baseline, item_item, svd], 0.01]
    return (count_dict, values)

############################################## TABLE_2 #########################################################

def table_2(changes_dict, count_dict):
  changes_dict = {id:list for id,list in changes_dict.items() if len(list)>1}
  patterns = {}
  for id, list in changes_dict.items():
    new_list = []
    for alg in list:
      new_alg = LONG_TO_SHORT[alg]
      new_list.append(new_alg)
    recalg_tup = tuple(new_list)
    changes_dict[id] = recalg_tup
    patterns[recalg_tup] = 0
    
  for recalg_tup in changes_dict.values():
    patterns[recalg_tup] += 1
  
  patterns_found = set()

  for id, recalg_tup in changes_dict.items():
    initial = recalg_tup[0]
    count = 0
    if recalg_tup in patterns.keys():
      count = patterns[recalg_tup]
    if(count>7 and count<53):
      percentage = (count/count_dict[initial][1])*100
      pattern_info = ("pattern: {} | count: {} | percentage: {}".format(recalg_tup, count, percentage))
      patterns_found.add(pattern_info)
  
  for item in patterns_found:
    print(item)

################################################# FIG_4 ########################################################

def fig_4(changes_dict, range):
    transitions_dict = {}
    
    # range check -- running it on data where the range is outside of 20
    # accepting the broadest possible input | returning the most specific answer
    # for k in range(20):
    #     transitions_dict[k+1] = 0
    lower_bound = range[0]
    upper_bound = range[1]
    for list in changes_dict.values():
        transition_count = len(list) - 1
        if transition_count>lower_bound and transition_count<upper_bound: 
            transitions_dict[transition_count] = 1 + transitions_dict.get(transition_count, 0)
    return transitions_dict

######################################## SSH_CONNECTION #################################################

def SSH_connection(query):
  with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
    conn = pymysql.connect(host='127.0.0.1', user=sql_username,
                            passwd=sql_password, db=sql_main_database,
                            port=tunnel.local_bind_port)
    print('connection done')
    df_result = pd.read_sql_query(query,conn)
  
  return df_result

#################################################################################################################

In [8]:
# CALCULATING NUMBER OF USERS 
query_users = ''' SELECT expt_user.group, expt_user.userId from ML3_mirror.expt_user WHERE 
                exptId = 'listcmp-long' AND started <'2015-04-01' 
                AND status = 'RANDOMLY_ASSIGNED' '''
users = SSH_connection(query_users)
print("Users: {}".format(users.shape[0]))

connection done
Users: 3005


In [16]:
# NUMBER OF CHANGE EVENTS 
query_change = '''SELECT userId, logJson, tstamp, action from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED') 
                AND log_action.action='recommender-change' 
                AND log_action.tstamp<'2015-04-01' AND log_action.tstamp>='2014-11-04' ORDER BY log_action.tstamp'''
change_events = SSH_connection(query_change)
print("query done")
print(change_events.shape[0])

connection done
query done
12009


In [17]:
print(change_events['tstamp'].min())
print(change_events['tstamp'].max())

2014-11-04 14:55:18
2015-03-31 21:37:24


In [ ]:
# SETTING UP DICTS 
dicts = primary_dicts(users, change_events, ('userId', 'group', 'logJson'))
changes_dict = dicts[0]
recchange_dict = dicts[1]


# TABLE 1
print("Recommender change events: ", sum(changes_dict.values()))
changes_dict = {key:val for key,val in changes_dict.items() if val !=0}
print("Users switching at least once: ", len(changes_dict.values()))
print("Median changes per user w/ at least 1 change: ", int(np.median(list(changes_dict.values()))))

In [ ]:
# FIGURE 2 
func_out = fig_2(recchange_dict, changes_dict)
values = func_out[1]
count_dict = func_out[0]
draw_graph(values, ["Initial Algorithm", "Proportion of Users Switching"], [0, 0.35], [0, 0.1])

In [ ]:
# TABLE 2 
table_2(recchange_dict, count_dict)

In [ ]:
# FIGURE 3
for key in count_dict.keys():
    count_dict[key] = 0

for userId, recalg_tup in recchange_dict.items():
    if(userId in changes_dict.keys()):
        alg = LONG_TO_SHORT[recalg_tup[-1]]
        count_dict[alg] += 1

values = [["baseline", "item-item", "pick-groups", "svd"], 
        [count_dict["baseline"], count_dict["item-item"], 
        count_dict["pick-groups"], count_dict["svd"]], 
        8]
draw_graph(values, ["Final Selected Algorithm", "# of users"], [0, 400], [0, 100])

# FIGURE 4
func_out = fig_4(recchange_dict, (0, 20))
plt.hist(func_out.keys(), weights=func_out.values(), bins = range(21))
plt.show()

In [118]:
query_time = '''SELECT count(*) from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED')
                AND log_action.tstamp<'2015-04-01' ORDER BY userId'''
# query_time = ''' SELECT userId,tstamp from ML3_mirror.log_action limit 10 '''
transition_times = SSH_connection(query_time)
print(transition_times)
# rows = transition_times.shape[0]

# 621277 rows

connection done
   count(*)
0    621277


In [9]:
import datetime

In [18]:
def remove_items(dict_in, desired_diff):
    dict_diffs = dict_in
    for key,val in dict_diffs.items():
        if val>desired_diff:
            dict_diffs.pop(key)
    return len(dict_diffs.keys())

In [ ]:
query_time = '''SELECT userId, action, tstamp, logJson from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED')
                AND log_action.tstamp<'2015-04-01' AND log_action.tstamp>='2014-11-04' ORDER BY log_action.tstamp'''
user_actions = SSH_connection(query_time)

In [41]:
user_actions.head(210)
# print(user_actions.index.is_monotonic_increasing)
# pd.algos.is_monotonic_int64(user_actions['userId'].values, False)[0]

,userId,action,tstamp
0,251465,pageview,2014-11-04 14:53:16
1,251465,pageview,2014-11-04 14:53:46
2,251465,pageview,2014-11-04 14:54:13
3,251465,recommender-change,2014-11-04 14:55:18
4,251465,pageview,2014-11-04 14:55:19
...,...,...,...
205,247359,pageview,2014-11-04 18:15:11
206,247359,pageview,2014-11-04 18:15:32
207,169946,pageview,2014-11-04 18:15:46
208,169946,pageview,2014-11-04 18:16:02


In [ ]:
query = '''SELECT userId, action, logJson, tstamp from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED')
                AND log_action.tstamp<'2015-04-01' AND log_action.tstamp>='2014-11-04' ORDER BY log_action.userId, log_action.tstamp'''
check = SSH_connection(query)
print(check.shape[0])
# print(check)

In [43]:
check.head(210)

,userId,loginId,action,tstamp
0,15226,2AXIIgi,pageview,2015-01-06 08:12:23
1,15226,2AXIIgi,pageview,2015-01-06 08:12:34
2,15226,2AXIIgi,pageview,2015-01-06 08:13:10
3,15226,2AXIIgi,pageview,2015-01-06 08:13:23
4,15226,2AXIIgi,pageview,2015-01-06 08:13:37
...,...,...,...,...
205,15226,l8s4Bs2,pageview,2015-01-26 21:53:48
206,15226,l8s4Bs2,pageview,2015-01-26 21:53:50
207,15226,l8s4Bs2,rating,2015-01-26 21:53:55
208,15226,l8s4Bs2,pageview,2015-01-26 21:54:04


In [16]:
query_users = ''' SELECT userId, MIN(tstamp) as tstamp from ML3_mirror.user_login WHERE EXISTS 
                  (SELECT userId from ML3_mirror.expt_user where expt_user.userId = user_login.userId 
                  AND expt_user.exptId='listcmp-long' 
                  AND expt_user.status='RANDOMLY_ASSIGNED') GROUP BY userId '''
start_times_raw = SSH_connection(query_users)
start_times_raw.head()

connection done


,userId,tstamp
0,12559,2015-07-01 23:56:16
1,15226,2015-01-06 08:12:22
2,16783,2015-03-11 15:30:34
3,26229,2014-11-05 19:35:48
4,29340,2014-12-16 18:20:00


In [11]:
query = ''' SELECT userId, MIN(tstamp) as tstamp from ML3_mirror.log_action WHERE EXISTS 
            (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
            AND expt_user.exptId='listcmp-long' 
            AND expt_user.status='RANDOMLY_ASSIGNED') 
            AND log_action.tstamp<'2015-04-01' AND log_action.tstamp>='2014-11-04' GROUP BY userId '''
first_logs = SSH_connection(query)
first_logs.head()

connection done


,userId,tstamp
0,15226,2015-01-06 08:12:23
1,16783,2015-03-11 15:30:34
2,26229,2014-11-05 19:35:48
3,29340,2014-12-16 18:20:00
4,32451,2014-11-06 12:24:22


In [20]:
first_logs.shape[0] # 2999
start_times_raw.shape[0] # 3776

3776

In [19]:
first_logs_dict = first_logs.set_index('userId').to_dict()['tstamp']
change_events.head(20)

,userId,logJson,tstamp,action
0,251465,"{""engineId"":""pick-groups""}",2014-11-04 14:55:18,recommender-change
1,251465,"{""engineId"":""svd""}",2014-11-04 14:55:41,recommender-change
2,247359,"{""engineId"":""item-item""}",2014-11-04 18:05:30,recommender-change
3,169946,"{""engineId"":""pick-groups""}",2014-11-04 18:10:43,recommender-change
4,169946,"{""engineId"":""svd""}",2014-11-04 18:11:31,recommender-change
5,244993,"{""engineId"":""baseline""}",2014-11-04 18:42:20,recommender-change
6,244993,"{""engineId"":""item-item""}",2014-11-04 20:15:44,recommender-change
7,251201,"{""engineId"":""item-item""}",2014-11-05 01:22:15,recommender-change
8,252151,"{""engineId"":""pick-groups""}",2014-11-05 01:56:18,recommender-change
9,252263,"{""engineId"":""baseline""}",2014-11-05 02:31:05,recommender-change


In [43]:
actual_changes = primary_dicts(users, change_events, ('userId', 'group', 'logJson', 'tstamp'), first_logs_dict)
tstamps_dict = actual_changes[1]
count = actual_changes[0]


# k = 0
# for key,val in tstamps_dict.items():
#     if k<30 :
#         print(key,val)
#         k += 1
#     else:
#         break
print(len(tstamps_dict.keys()))
# print(len(next_try.keys()))
print(len(count.keys()))

221
747


In [21]:
print(user_actions['tstamp'].min())
print(user_actions['tstamp'].max())

# started_at = {}
total_time = {}
other_users = set()
rows = user_actions.shape[0]
print(rows)
# for k in range(rows):
#     userId = no_of_users[k].at['userId']
#     started_at[userId] = start_times_raw[k].at['tstamp']
# print(len(started_at.keys())) #2999

# actual_changes = primary_dicts(users, user_actions, ('userId', 'group', 'logJson', 'tstamp'))
# tstamps_dict = actual_changes[1]

# for key,val in tstamps_dict.items():
#         if key in first_logs_dict:
#                 diff = val - first_logs_dict[key]

#                 if diff.total_seconds()<3600:
#                         total_time[key] = diff.total_seconds()
#                 if diff.total_seconds()>3600:
#                         total_time[key] = diff.total_seconds()
#                         other_users.add(key)



# for k in range(rows):
#     userId = user_actions.loc[k].at['userId']
#     if user_actions.loc[k].at['action'] == 'recommender-change':
#         start_time_row = first_logs[first_logs['userId'] == userId]
#         start_time_tstamp = start_time_row.loc[start_time_row.index[0], 'tstamp']
#         diff = user_actions.loc[k].at['tstamp'] - start_time_tstamp 
#         # total_time[userId] = diff.total_seconds()
        
#         if diff.total_seconds()<3600 :
#             total_time[userId] = diff.total_seconds()
#         if diff.total_seconds()>3600:
#             total_time[userId] = diff.total_seconds()
#             other_users.add(userId)
        # if diff.total_seconds()>3600 and userId in total_time:
        #     total_time.pop(userId)

# for key in total_time.keys():
#     if key in other_users:
#         total_time.pop(key)

total_time = {key:val for key,val in total_time.items() if key not in other_users}

# len = remove_items(total_time, 3600)
# print(len)

# k=0
# for key,val in total_time.items():
#     if k <25:
#         print(key, val)
#     else:
#         break
#     k += 1
#536
#778
#5m 34.7s
print(len(total_time))
print(len(total_time)/748)
print(max(total_time.values()))
print(min(total_time.values()))
# ORDER BY timestamp 
# how would you do this for one user sorted by timestamp?
# what is the precise output for this problem? what is the output -- to represent the results?



2014-11-04 14:53:16
2015-03-31 23:49:59
621189
333
0.4451871657754011
3426.0
17.0


In [44]:
query_sessions = ''' SELECT userId, logJson, tstamp, action from ML3_mirror.log_action WHERE EXISTS 
                (SELECT userId from ML3_mirror.expt_user where expt_user.userId = log_action.userId 
                AND expt_user.exptId='listcmp-long' 
                AND expt_user.status='RANDOMLY_ASSIGNED')
                AND log_action.tstamp<'2015-04-01' AND log_action.tstamp>='2014-11-04' ORDER BY log_action.tstamp ''' 
logs = SSH_connection(query_sessions)
print(logs.shape[0])
logs.head()

#5m 35.9s

connection done
621189


,userId,logJson,tstamp,action
0,251465,"{""pageName"":""restricted.listcmpLongIntro"",""par...",2014-11-04 14:53:16,pageview
1,251465,"{""pageName"":""base.frontpage"",""params"":{}}",2014-11-04 14:53:46,pageview
2,251465,"{""pageName"":""base.pickGroups"",""params"":{}}",2014-11-04 14:54:13,pageview
3,251465,"{""engineId"":""pick-groups""}",2014-11-04 14:55:18,recommender-change
4,251465,"{""pageName"":""base.frontpage"",""params"":{}}",2014-11-04 14:55:19,pageview


In [109]:
def session_count(user_logs, tstamps_dict, fields):
    id_field = fields[0]
    time_field = fields[1]
    action_field = fields[2]

    sessions_dict = {}

    keys = tstamps_dict.keys()
    rec_changes_dict = dict.fromkeys(keys, [0])

    # k=0
    # for key,val in rec_changes_dict.items():
    #     if k<10:
    #         print(key,val)
    #         k += 1
    #     else:
    #         break
    test_var = 0
    for index,row in user_logs.iterrows():
        unique_id = row[id_field]
        action = row[action_field]

        diff = row[time_field] - tstamps_dict[unique_id]
        if diff.total_seconds() > 3600:
            sessions_dict[unique_id] = 1 + sessions_dict.get(unique_id, 1)
        else:
            sessions_dict[unique_id] = sessions_dict.get(unique_id, 1)
        
        if action == "recommender-change":
            session_num = sessions_dict[unique_id]


            val = rec_changes_dict[unique_id]
            # print(session_num - 1)
            # val[0] += 1
            rec_changes_dict[unique_id][0] = 1 + rec_changes_dict[unique_id][0]
            # test_var += 1
            # if len(val) == session_num - 1:
            #     rec_changes_dict[unique_id].append(1)
            # elif len(val)<session_num -1:
            #     val[len(val):session_num-1] = [0] * ((session_num)-len(val))
            #     val.append(1)
            #     rec_changes_dict[unique_id] = val
            # else:
            #     # print(val[session_num - 1])
            #     rec_changes_dict[unique_id][session_num - 1] += 1


            # rec_changes_dict[unique_id][session_num - 1] = 1 + rec_changes_dict.get(unique_id, 1)

        tstamps_dict[unique_id] = row[time_field]
    
    return (sessions_dict, rec_changes_dict, test_var)

In [ ]:
dictionaries = session_count(logs, first_logs_dict, ('userId', 'tstamp', 'action'))
recchanges = dictionaries[1]
sessions = dictionaries[0]
k=0
for key,val in recchanges.items():
    if k<25:
        print(key,val)
        k += 1
    else:
        break
# print(len(recchanges))
print(dictionaries[2])

In [93]:
my_dict = {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0]}
my_list = [0,1,2,3]

for item in my_list:
    if item == 1:
        my_dict[item][1] = item + 1

print(my_dict.items())


dict_items([(0, [0, 0]), (1, [0, 2]), (2, [0, 0]), (3, [0, 0])])


In [30]:
user_times_sorted = user_actions.sort_values(by=['userId', 'tstamp'])
# user_times_sorted.head(25)
print(user_times_sorted.shape[0])

621189


In [ ]:
# user_times_sorted = user_actions.sort_values(by=['userId', 'tstamp'])

curr_tstamps = {}
rec_change_sessions = {}

curr_tstamps = start_times_raw.set_index('userId').to_dict()['tstamp']

curr_userId = user_times_sorted.loc[0].at['userId']
curr_tstamp = user_times_sorted.loc[0].at['tstamp']

# for i in range(rows):
    
    


        


In [48]:
#TEST
start = time.time()
curr_tstamps = start_times_raw.set_index('userId').to_dict()['tstamp']
print("one liner: ", time.time() - start)

start = time.time()
start_tstamp_rows = start_times_raw.shape[0]
for k in range(start_tstamp_rows):
    userId = start_times_raw.loc[k].at['userId']
    start_tstamp = start_times_raw.loc[k].at['tstamp']
    curr_tstamps[userId] = start_tstamp
print("loop: ", time.time() - start)

one liner:  0.0068662166595458984
loop:  0.3140242099761963
